In [1]:
import os
os.chdir("../expand_latex_macros/")
import source
import regex as re

import importlib
importlib.reload(source)

<module 'source' from '/work/submit/mcgreivy/expand-latex-macros/expand_latex_macros/source.py'>

In [5]:
test_str = r"""\section{Introduction}
\label{sec:introduction}
While the underlying interactions of hadronic collisions and hadronisation are understood within the Standard Model, exact computation of the processes governed by QCD are difficult due to the highly non-linear nature of the strong force.  In the absence of full calculations, generators based on phenomenological models have been devised and optimised, or ``tuned'', to accurately reproduce experimental observations.  These generators predict how Standard Model physics will behave at the LHC  and constitute the reference for discoveries of New Physics effects.  
Strange quark production is a powerful probe for hadronisation processes at  { {p-p}} colliders since protons have no net strangeness.  Recent experimental results in the field have been published by STAR ~\cite{STAR} from RHIC   { {p-p}} collisions at  { s }  = 0.2 {  {TeV}} and by ALICE ~\cite{ALICE}, CMS ~\cite{CMS} and LHCb ~\cite{KsCrossSection} from LHC   { {p-p}} collisions at  { s }  = 0.9 and 7 {  {TeV}}.  LHCb  can make an important contribution thanks to a full instrumentation of the detector in the forward region that is unique among the LHC  experiments.  Studies of data recorded at different energies with the same apparatus help to control the experimental systematic uncertainties.  
In this paper we report on measurements of the efficiency corrected production ratios of the strange particles  {\overline  { {\Lambda} } } ,  { {\Lambda} }  and  { { K } ^0_{   S}}  as observables related to the fundamental processes behind parton fragmentation and hadronisation.  The ratios
\begin{align}
\frac{ {\overline  { {\Lambda} } } }{ { {\Lambda} } } &=\frac{\sigma( { { {p-p}}  {\rightarrow}   {\overline  { {\Lambda} } }  X} )}{\sigma( { { {p-p}}  {\rightarrow}   { {\Lambda} }  X} )} 
\intertext{and}
\frac{ {\overline  { {\Lambda} } } }{ { { K } ^0_{   S}} } &=\frac{\sigma( { { {p-p}}  {\rightarrow}   {\overline  { {\Lambda} } }  X} )}{\sigma( { { {p-p}}  {\rightarrow}   { { K } ^0_{   S}}  X} )}
\end{align}
have predicted dependences on rapidity,  y , and transverse momentum,  {p_T} , which can vary strongly between different tunes of the generators.  
Measurements of the ratio  {\overline  { {\Lambda} } } / { {\Lambda} }  allow the study of the transport of baryon number from  { {p-p}} collisions to final state hadrons and the ratio  {\overline  { {\Lambda} } } / { { K } ^0_{   S}}  is a measure of baryon-to-meson suppression in strange quark hadronisation.
\section{The LHCb detector and data samples}
\label{sec:detector}
The Large Hadron Collider beauty experiment (LHCb) at CERN is a single-arm spectrometer covering the forward rapidity region.  The analysis presented in this paper relies exclusively on the tracking detectors.  The high precision tracking system begins with a silicon strip Vertex Locator (VELO), designed to identify displaced secondary vertices up to about 65 {   cm}  downstream of the nominal interaction point.  A large area silicon tracker follows upstream of a dipole magnet and tracker stations, built with a mixture of straw tube and silicon strip detectors, are located downstream.  The LHCb  coordinate system is defined to be right-handed with its origin at the nominal interaction point, the z axis aligned along the beam line towards the magnet and the y axis pointing upwards.  The bending plane is horizontal and the magnet has a reversible field, with the positive B_y polarity called ``up'' and the negative ``down''.  Tracks reconstructed through the full spectrometer experience an integrated magnetic field of around 4 Tm.  The detector is described in full elsewhere \cite{DetectorPaper}.
A loose minimum bias trigger is used for this analysis, requiring at least one track segment in the downstream tracking stations.  This trigger is more than 99 \% efficient for offline selected events that contain at least two tracks reconstructed through the full system.
Complementary data sets were recorded at two collision energies of  { s }  = 0.9 and 7 {  {TeV}}, with both polarities of the dipole magnet.  An integrated luminosity of  {0.3  {nb}^{-1}}  (corresponding to 12.5 million triggers) was taken at the lower energy, of which 48 \% had the up  magnetic field configuration.  At the higher energy, 67 \% of a total  {1.8  {nb}^{-1}}  (110.3 million triggers) was taken with field up .
At injection energy ( { {\sqrts}  = 0.9  { { Te  -0.1em V}} } ), the proton beams are significantly broadened spatially compared to the accelerated beams at  { {\sqrts}  = 7  { { Te  -0.1em V}} } .  To protect the detector, the two halves of the VELO are retracted along the x axis from their nominal position of inner radius of 8 {   mm}  to the beam, out to 18 {   mm} , which results in a reduction of the detector acceptance at small angles to the beam axis by approximately 0.5 units of rapidity.
The beams collide with a crossing angle in the horizontal plane tuned to compensate for LHCb 's magnetic field.  The angle required varies as a function of beam configuration and for the data taking period covered by this study was set to 2.1 {   mrad}  at  { {\sqrts}  = 0.9  { { Te  -0.1em V}} }  and 270 { { \textmu rad}}  at 7 {  {TeV}}.  Throughout this analysis  { V ^0}   momenta and any derived quantity such as rapidity are computed in the centre-of-mass frame of the colliding protons.
Samples of Monte Carlo (MC) simulated events have been produced in close approximation to the data-taking conditions described above for estimation of efficiencies and systematic uncertainties.  A total of 73 million simulated minimum bias events were used for this analysis per magnet polarity at  { {\sqrts}  = 0.9  { { Te  -0.1em V}} }  and 60 (69) million events at 7 {  {TeV}} for field up  (down ).  LHCb  MC simulations are described in Ref.~\cite{Gauss}, with  { {p-p}} collisions generated by  { {Pythia}}  6 \cite{PYTHIA6}.  Emerging particles decay via  { {EvtGen}}  \cite{EvtGen}, with final state radiation handled by  { {Photos}}  \cite{PHOTOS}.  The resulting particles are transported through LHCb  by  { {Geant}}  \cite{Geant}, which models hits on the sensitive elements of the detector as well as interactions between the particles and the detector material.  Secondary particles produced in these material interactions decay via  { {Geant}} .
Additional samples of five million minimum bias events were generated for studies of systematic uncertainties using  { {Pythia}}  6 variants Perugia 0 (tuned on experimental results from SPS, LEP and Tevatron) and Perugia NOCR (an extreme model of baryon transport) \cite{Peter}.  Similarly sized samples of  { {Pythia}}  8 \cite{PYTHIA8} minimum bias diffractive events were also generated, including both hard and soft diffraction \footnote{Single- and double-diffractive process types are considered: 92--94 in  { {Pythia}}  6.421, with soft diffraction, and 103--105 in  { {Pythia}}  8.130, with soft and hard diffraction.} \cite{Navin}.
\section{Analysis procedure}
\label{sec:analysis}
{ V ^0}   hadrons are named after the ``V''-shaped track signature of their dominant decays:  { { { {\Lambda} }   {\rightarrow}   { p }   { { {\pi} } ^-} } } ,  { { {  0.2em \overline{  -0.2em  { {\Lambda} }   0.05em} -0.05em }   {\rightarrow}   {\overline  { p } }   { { {\pi} } ^+} } }  and  { { { { K } ^0_{   S}}   {\rightarrow}   { { {\pi} } ^+}   { { {\pi} } ^-} } } , which are reconstructed for this analysis.  Only tracks with quality  {\chi^2}  /  {ndf} <9 are considered, with the  { V ^0}   required to decay within the VELO and the daughter tracks to be reconstructed through the full spectrometer.  Any oppositely-charged pair is kept as a potential  { V ^0}   candidate if it forms a vertex with  {\chi^2}  < 9 (with one degree of freedom for a  { V ^0}   vertex).   {\overline  { {\Lambda} } } ,  { {\Lambda} }  and  { { K } ^0_{   S}}  candidates are required to have invariant masses within \pm50 {  {MeV}  -2mu/  -1mu c^2} of the PDG values \cite{PDG}.  This mass window is large compared to the measured mass resolutions of about 2 {  {MeV}  -2mu/  -1mu c^2} for  { {\Lambda} }  ( {\overline  { {\Lambda} } } ) and 5 {  {MeV}  -2mu/  -1mu c^2} for  { { K } ^0_{   S}} .
Combinatorial background is reduced with a Fisher discriminant based on the impact parameters (IP) of the daughter tracks (d^{\pm}) and of the reconstructed  { V ^0}   mother, where the impact parameter is defined as the minimum distance of closest approach to the nearest reconstructed primary interaction vertex measured in mm.  The Fisher discriminant:
\begin{equation}
{\mathcalF_{ {IP}}}  = a \log_{10}(d^{+}_ {IP}/1  {mm}) + b \log_{10}(d^{-}_ {IP}/1  {mm}) + c \log_{10}( { V ^0}  _ {IP}/1  {mm})
\end{equation}
is optimised for signal significance (S/\sqrt{S+B}) on simulated events after the above quality criteria.  The cut value,  {\mathcalF_{ {IP}}} >1, and coefficients, a=b=-c=1, were found to be suitable for  {\overline  { {\Lambda} } } ,  { {\Lambda} }  and  { { K } ^0_{   S}}  at both collision energies (Fig.~\ref{fig:fisher}).
\begin{figure}
\subfigure[]{
\includegraphics[width=0.47 \textwidth]{figs/Fig1a}
\label{fig:fisher-k}
}
\subfigure[]{
\includegraphics[width=0.47 \textwidth]{figs/Fig1b}
\label{fig:fisher-a}
}
\caption{  The Fisher discriminant  {\mathcalF_{ {IP}}}  in 0.5 million Monte Carlo simulated minimum bias events at  { {\sqrts}  = 7  { { Te  -0.1em V}} }  for \subref{fig:fisher-k}  { { K } ^0_{   S}}  and \subref{fig:fisher-a}  {\overline  { {\Lambda} } } .}
\label{fig:fisher}
\end{figure}
The  { {\Lambda} }  ( {\overline  { {\Lambda} } } ) signal significance is improved by a \pm4.5 {  {MeV}  -2mu/  -1mu c^2} veto around the PDG  { { K } ^0_{   S}}  mass after re-calculation of each candidate's invariant mass with an alternative  { { {\pi} } ^+ { {\pi} } ^-}  daughter hypothesis.  A similar veto to remove  { {\Lambda} }  ( {\overline  { {\Lambda} } } ) with a  { p }  { { {\pi} } ^-}  ( {\overline  { p } }  { { {\pi} } ^+} ) hypothesis from the  { { K } ^0_{   S}}  sample is not found to improve significance so is not applied.
After the above selection,  { V ^0}   yields are estimated from data and simulation by fits to the invariant mass distributions, examples of which are shown in Fig.~\ref{fig:mass-fits}.  These fits are carried out with the method of unbinned extended maximum likelihood and are parametrised by a double Gaussian signal peak (with a common mean) over a linear background.  The mean values show a small, but statistically significant, deviation from the known  { { K } ^0_{   S}}  and  { {\Lambda} }  ( {\overline  { {\Lambda} } } ) masses \cite{PDG}, reflecting the status of the momentum-scale calibration of the experiment.  The width of the peak is computed as the quadratic average of the two Gaussian widths, weighted by their signal fractions.  This width is found to be constant as a function of  {p_T}  and increases linearly toward higher  y ,  {  e.g.}  by 1.4 (0.8) {  {MeV}  -2mu/  -1mu c^2} per unit rapidity for  { { K } ^0_{   S}}  ( { {\Lambda} }  and  {\overline  { {\Lambda} } } ) at  { {\sqrts}  = 7  { { Te  -0.1em V}} } .  The resulting signal yields are listed in Table~\ref{tab:signal-yields}.
Significant differences are observed between  { V ^0}   kinematic variables reconstructed in data and in the simulation used for efficiency determination.  These differences can produce a bias for the measurement of  {\overline  { {\Lambda} } } / { { K } ^0_{   S}}  given the different production kinematics of the baryon and meson.  Simulated  { V ^0}   candidates are therefore weighted to match the two-dimensional  {p_T} ,  y  distributions observed in data. These distributions are shown projected along both axes in Fig.~\ref{fig:weighting}.  The  { V ^0}   signal yield  {p_T} ,  y  distributions are estimated from selected data and Monte Carlo candidates using sideband subtraction.  Two-dimensional fits, linear in both  {p_T}  and  y , are made to the ratios data/MC of these yields independently for  {\overline  { {\Lambda} } } ,  { {\Lambda} }  and  { { K } ^0_{   S}} , for each magnet polarity and collision energy.  The resulting functions are used to weight generated and selected  { V ^0}   candidates in the Monte Carlo simulation.  These weights vary across the measured  {p_T} ,  y  range between 0.4 and 2.1, with typical values between 0.8 and 1.2.  
\begin{figure}[t!]
\subfigure[]{
\includegraphics[width=0.47 \textwidth]{figs/Fig2a}
\label{fig:mass-fit-a}
}
\subfigure[]{
\includegraphics[width=0.47 \textwidth]{figs/Fig2b}
\label{fig:mass-fit-k}
}
\caption{  Invariant mass peaks for \subref{fig:mass-fit-a}  {\overline  { {\Lambda} } }  in the range 0.25< {p_T} <2.50 {  {GeV}  -2mu/  -1mu c} \& 2.5< y <3.0 and \subref{fig:mass-fit-k}  { { K } ^0_{   S}}  in the range 0.65< {p_T} <1.00 {  {GeV}  -2mu/  -1mu c} \& 3.5< y <4.0 at  { {\sqrts}  = 0.9  { { Te  -0.1em V}} }  with field up .  Signal yields, N, are found from fits (solid curves) with a double Gaussian peak with common mean, \mu, over a linear background (dashed lines).  The width, \sigma, is computed as the quadratic average of the two Gaussian widths weighted by their signal fractions.}
\label{fig:mass-fits}
\end{figure}
\begin{table}
\caption{  Integrated signal yields extracted by fits to the invariant mass distributions of selected  { V ^0}   candidates from data taken with magnetic field up  and down  at  { s }  = 0.9 and 7 {  {TeV}}.}
\\
\begin{tabular}{c|cc|cc}
\toprule
{ s }  & \multicolumn2{c|}{0.9 {  {TeV}}} & \multicolumn2c{7 {  {TeV}}} \\
Magnetic field  & Up & Down & Up & Down \\
\midrule
{\overline  { {\Lambda} } }   &  3,440 \pm60  &  4,100 \pm70  & 258,930 \pm640 & 132,550 \pm460 \\
{ {\Lambda} }      &  4,880 \pm80  &  5,420 \pm80  & 294,010 \pm680 & 141,860 \pm460 \\
{ { K } ^0_{   S}}     & 35,790 \pm200 & 40,230 \pm220 & 2,737,090 \pm1,940 & 1,365,990 \pm1,370 \\
\bottomrule
\end{tabular}
\label{tab:signal-yields}
\end{table}
The measured ratios are presented in three complementary binning schemes: projections over the full  {p_T}  range, the full  y  range, and a coarser two-dimensional binning.  The rapidity range  {2.0 <  y  < 4.0 (4.5)} is split into 0.5-unit bins, while six bins in  {p_T}  are chosen to approximately equalise signal  { V ^0}   statistics in data over the range  {0.25 (0.15) <  {p_T}  < 2.50} {  {GeV}  -2mu/  -1mu c} from collisions at  { s }  = 0.9 (7) {  {TeV}}.  The two-dimensional binning combines pairs of  {p_T}  bins. The full analysis procedure is carried out independently in each  {p_T} ,  y  bin.
The efficiency for selecting prompt  { V ^0}   decays is estimated from simulation as
\begin{equation} \label{equ:efficiency}
\varepsilon = \frac{ N( { V ^0}    {\rightarrow}  d^{+}d^{-})_{ {Observed}} }{ N(pp  {\rightarrow}   { V ^0}   X)_{ {Generated}} },
\end{equation}
where the denominator is the number of prompt  { V ^0}   hadrons generated in a given  {p_T} ,  y  region after weighting and the numerator is the number of those weighted candidates found from the selection and fitting procedure described above.  The efficiency therefore accounts for decays via other channels and losses from interactions with the detector material.  Prompt  { V ^0}   hadrons are defined in Monte Carlo simulation by the cumulative lifetimes of their ancestors
\begin{equation} \label{equ:prompt-cut}
\sum \limits_{i=1}^n c \tau_i < 10^{-9} {   m} ,
\end{equation}
where \tau_i is the proper decay time of the i^{ {th}} ancestor.  This veto is defined such as to keep only  { V ^0}   hadrons created either directly from the  { {p-p}} collisions or from the strong or electromagnetic decays of particles produced at those collisions, removing  { V ^0}   hadrons generated from material interactions and weak decays.  The Fisher discriminant  {\mathcalF_{ {IP}}}  strongly favours prompt  { V ^0}   hadrons, however a small non-prompt contamination in data would lead to a systematic bias in the ratios.  The fractional contamination of selected events is determined from simulation to be 2-6 \% for  {\overline  { {\Lambda} } }  and  { {\Lambda} } , depending on the measurement bin, and about 1 \% for  { { K } ^0_{   S}} .  This effect is dominated by weak decays rather than material interactions.  The resulting absolute corrections to the ratios  {\overline  { {\Lambda} } } / { {\Lambda} }  and  {\overline  { {\Lambda} } } / { { K } ^0_{   S}}  are approximately 0.01.
\begin{figure}
\subfigure[]{
\includegraphics[width=0.47 \textwidth]{figs/Fig3a}
\label{fig:weighting-pt}
}
\subfigure[]{
\includegraphics[width=0.47 \textwidth]{figs/Fig3b}
\label{fig:weighting-y}
}
\caption{  \subref{fig:weighting-pt} Transverse momentum and \subref{fig:weighting-y} rapidity distributions for  { { K } ^0_{   S}}  in data and Monte Carlo simulation at  { {\sqrts}  = 7  { { Te  -0.1em V}} } .  The difference between data and Monte Carlo is reduced by weighting the simulated candidates.}
\label{fig:weighting}
\end{figure}
\section{Systematic uncertainties}\label{sec:systematics}
The measured efficiency corrected ratios  {\overline  { {\Lambda} } } / { {\Lambda} }  and  {\overline  { {\Lambda} } } / { { K } ^0_{   S}}  are subsequently corrected for non-prompt contamination as found from Monte Carlo simulation and defined by Eq.~\ref{equ:prompt-cut}.  This procedure relies on simulation and the corrections may be biased by the choice of the LHCb  MC generator tune.  To estimate a systematic uncertainty on the correction for non-prompt  { V ^0}  , the contaminant fractions are also calculated using two alternative tunes of  { {Pythia}}  6: Perugia 0 and Perugia NOCR \cite{Peter}.  The maximum differences in non-prompt fraction across the measurement range and at both energies are <1 \% for each  { V ^0}   species.  The resulting absolute uncertainties on the ratios are <0.01.
The efficiency of primary vertex reconstruction may introduce a bias on the measured ratios if the detector occupancy is different for events containing  { { K } ^0_{   S}} ,  { {\Lambda} }  or  {\overline  { {\Lambda} } } .  This efficiency is compared in data and simulation using  { V ^0}   samples obtained with an alternative selection not requiring a primary vertex.  Instead, the  { V ^0}   flight vector is extrapolated towards the beam axis to find the point of closest approach.  The z coordinate of this point is used to define a pseudo-vertex, with x=y=0.  Candidates are kept if the impact parameters of their daughter tracks to this pseudo-vertex are >0.2 mm.  There is a large overlap of signal candidates with the standard selection.  The primary vertex finding efficiency is then explored by taking the ratio of these selected events which do or do not have a standard primary vertex.  Calculated in bins of  {p_T}  and  y , this efficiency agrees between data and simulation to better than 2 \% at both  { s }  = 0.9 and 7 {  {TeV}}. The resulting absolute uncertainties on  {\overline  { {\Lambda} } } / { {\Lambda} }  and  {\overline  { {\Lambda} } } / { { K } ^0_{   S}}  are <0.02 and <0.01, respectively.
\begin{figure}
\subfigure[]{
\includegraphics[width = 0.48 \textwidth]{figs/Fig4a}
\label{fig:material-int-al}
}
\subfigure[]{
\includegraphics[width = 0.48 \textwidth]{figs/Fig4b}
\label{fig:material-int-ak}
}
\caption{  The double ratios \subref{fig:material-int-al} ( {\overline  { {\Lambda} } } / { {\Lambda} } )_ {Data}/( {\overline  { {\Lambda} } } / { {\Lambda} } )_ {MC} and \subref{fig:material-int-ak}  ( {\overline  { {\Lambda} } } / { { K } ^0_{   S}} )_ {Data}/( {\overline  { {\Lambda} } } / { { K } ^0_{   S}} )_ {MC} are shown as a function of the material traversed, in units of radiation length.  Flat line fits, shown together with their respective \chi^2 probabilities, give no evidence of a bias.}
\label{fig:material-int-ratios}
\end{figure}
The primary vertex finding algorithm requires at least three reconstructed tracks.\footnote{The minimum requirements for primary vertex reconstruction at LHCb  can be approximated in Monte Carlo simulation by a generator-level cut requiring at least three charged particles from the collision with lifetime c \tau > 10^{-9} {   m} , momentum p>0.3 {  {GeV}  -2mu/  -1mu c} and polar angle 15<\theta<460 {   mrad} .}  Therefore, the reconstruction highly favours non-diffractive events due to the relatively low efficiency for finding diffractive interaction vertices, which tend to produce fewer tracks.  In the LHCb  MC simulation, the diffractive cross-section accounts for 28 (25) \% of the total minimum-bias cross-section of 65 (91) mb at 0.9 (7) {  {TeV}} \cite{Gauss}.  Due to the primary vertex requirement, only about 3 \% of the  { V ^0}   candidates selected in simulation are produced in diffractive events.  These fractions are determined using  { {Pythia}} ~6 which models only soft diffraction.  As a cross check, the fractions are also calculated with  { {Pythia}} ~8 which includes both soft and hard diffraction.  The variation on the overall efficiency between models is about 2 \% for both ratios at  { {\sqrts}  = 7  { { Te  -0.1em V}} }  and close to 1 \% at 0.9 {  {TeV}}.  Indeed, complete removal of diffractive events only produces a change of 0.01-0.02 in the ratios across the measurement range.
The track reconstruction efficiency depends on particle momentum.  In particular, the tracking efficiency varies rapidly with momentum for tracks below 5 {  {GeV}  -2mu/  -1mu c}.  Any bias is expected to be negligible for the ratio  {\overline  { {\Lambda} } } / { {\Lambda} }  but can be larger for  {\overline  { {\Lambda} } } / { { K } ^0_{   S}}  due to the different kinematics.  Two complementary procedures are employed to check this efficiency.  First, track segments are reconstructed in the tracking stations upstream of the magnet.  These track segments are then paired with the standard tracks reconstructed through the full detector and the pairs are required to form a  { { K } ^0_{   S}}  to ensure only genuine tracks are considered.  This track matching gives a measure of the tracking efficiency for the upstream tracking systems.  The second procedure uses the downstream stations to reconstruct track segments, which are similarly paired with standard tracks to measure the efficiency of the downstream tracking stations.   The agreement between these efficiencies in data and simulation is better than 5 \%.  To estimate the resulting uncertainty on  {\overline  { {\Lambda} } } / { {\Lambda} }  and  {\overline  { {\Lambda} } } / { { K } ^0_{   S}} , both ratios are re-calculated after weighting  { V ^0}   candidates by 95 \% for each daughter track with momentum below 5 {  {GeV}  -2mu/  -1mu c}.  The resulting systematic shifts in the ratios are <0.01.
\begin{table}
\caption{  Absolute systematic errors are listed in descending order of importance.  Ranges indicate uncertainties that vary across the measurement bins and/or by collision energy.  Correlated sources of uncertainty between field up  and down  are identified.}
\\
\begin{tabular}{lcc}
\toprule
Sources of systematic uncertainty &  {\overline  { {\Lambda} } } / { {\Lambda} }  &  {\overline  { {\Lambda} } } / { { K } ^0_{   S}}  \\
\midrule
\emph{Correlated between field up  and down }: & & \\ 
Material interactions               & 0.02 & 0.02 \\ 
Diffractive event fraction          & 0.01-0.02 & 0.01-0.02 \\ 
Primary vertex finding              & <0.02 & <0.01 \\ 
Non-prompt fraction                 & <0.01  & <0.01  \\ 
Track finding                       & negligible & 0.01 \\
\midrule
\emph{Uncorrelated}: &  &  \\  
Kinematic correction             & 0.01-0.05 & <0.03 \\ 
Signal extraction from fit         & 0.001 & 0.001 \\
\midrule
Total & 0.02-0.06 & 0.02-0.03 \\ 
\bottomrule
\end{tabular}
\label{tab:syst-summary}
\end{table}
Particle interactions within the detector are simulated using the  { {Geant}}  package, which implements interaction cross-sections for each particle according to the LHEP physics list \cite{Geant}.  These simulated cross-sections have been tested in the LHCb  framework and are consistent with the LHEP values.  The small measured differences are propagated to  {\overline  { {\Lambda} } } / { {\Lambda} }  and  {\overline  { {\Lambda} } } / { { K } ^0_{   S}}  to estimate absolute uncertainties on the ratios of about 0.02.   { V ^0}   absorption is limited by the requirement that each  { V ^0}   decay occurs within the most upstream tracker (the VELO).  Secondary  { V ^0}   production in material is suppressed by the Fisher discriminant, which rejects  { V ^0}   candidates with large impact parameter.  The potential bias on the ratios is explored by measurement of both  {\overline  { {\Lambda} } } / { {\Lambda} }  and  {\overline  { {\Lambda} } } / { { K } ^0_{   S}}  as a function of material traversed (determined by the detector simulation), in units of radiation length, X_0.   Data and simulation are compared by their ratio, shown in Fig.~\ref{fig:material-int-ratios}.  These double ratios are consistent with a flat line as a function of X_0, therefore any possible imperfections in the description of the detector material in simulation do not have a large effect on the  { V ^0}   ratios. Note that the double ratios are not expected to be unity since simulations do not predict the same values for  {\overline  { {\Lambda} } } / { {\Lambda} }  and  {\overline  { {\Lambda} } } / { { K } ^0_{   S}}  as are observed in data.
The potential bias from the Fisher discriminant,  {\mathcalF_{ {IP}}} , is investigated using a pre-selected sample, with only the track and vertex quality cuts applied.  The distributions of  {\mathcalF_{ {IP}}}  for  { {\Lambda} } ,  {\overline  { {\Lambda} } }  and  { { K } ^0_{   S}}  in data and Monte Carlo simulation are estimated using sideband subtraction.  The double ratios of data/MC efficiencies are seen to be independent of the discriminant, implying that the distribution is well modelled in the simulation.  No systematic uncertainty is assigned to this selection requirement.
A degradation is observed of the reconstructed impact parameter resolution in data compared to simulation.  The simulated  { V ^0}   impact parameters are recalculated with smeared primary and secondary vertex positions to match the resolution measured in data.  There is a negligible effect on the  { V ^0}   ratio results.
A good estimate of the reconstructed yields and their uncertainties in both data and simulation is provided by the fitting procedure but there may be a residual systematic uncertainty from the choice of this method.  Comparisons are made using side-band subtraction and the resulting  { V ^0}   yields are in agreement with the results of the fits at the 0.1 \% level.  The resulting absolute uncertainties on the ratios are on the order of 0.001.
Simulated events are weighted to improve agreement between simulated  { V ^0}   kinematic distributions and data.  As described in Section~\ref{sec:analysis}, these weights are calculated from a two-dimensional fit, linear in both  {p_T}  and  y , to the distribution of the ratio between reconstructed data and simulated Monte Carlo candidates.  This choice of parametrisation could be a source of systematic uncertainty, therefore alternative procedures are investigated including a two-dimensional polynomial fit to 3^{ {rd}} order in both  {p_T}  and  y  and a (non-parametric) bilinear interpolation.  The results from each method are compared across the measurement range to estimate typical systematic uncertainties of 0.01-0.05 for  {\overline  { {\Lambda} } } / { {\Lambda} }  and <0.03 for  {\overline  { {\Lambda} } } / { { K } ^0_{   S}} .
The lifetime distributions of reconstructed and selected  { V ^0}   candidates are consistent between data and simulation.  The possible influence of transverse  { {\Lambda} }  ( {\overline  { {\Lambda} } } ) polarisation was explored by simulations with extreme values of polarisation and found to produce no significant effect on the measured ratios. Potential acceptance effects were checked as a function of azimuthal angle, with no evidence of systematic bias.  The potential sources of systematic uncertainty or bias are summarised in Table~\ref{tab:syst-summary}. 
\begin{figure}[t!]
\subfigure[]{
\includegraphics[width=0.47 \textwidth]{figs/Fig5a}
\label{fig:plotData-900-al-3y}
}
\subfigure[]{
\includegraphics[width=0.47 \textwidth]{figs/Fig5b}
\label{fig:plotData-7-al-3y}
}
\subfigure[]{
\includegraphics[width=0.47 \textwidth]{figs/Fig5c}
\label{fig:plotData-900-ak-3y}
}
\subfigure[]{
\includegraphics[width=0.47 \textwidth]{figs/Fig5d}
\label{fig:plotData-7-ak-3y}
}
\caption{  The ratios  {\overline  { {\Lambda} } } / { {\Lambda} }  and  {\overline  { {\Lambda} } } / { { K } ^0_{   S}}  from the full analysis procedure at \subref{fig:plotData-900-al-3y} \& \subref{fig:plotData-900-ak-3y}  { {\sqrts}  = 0.9  { { Te  -0.1em V}} }  and \subref{fig:plotData-7-al-3y} \& \subref{fig:plotData-7-ak-3y} 7 {  {TeV}} are shown as a function of rapidity, compared across intervals of transverse momentum.  Vertical lines show the combined statistical and systematic uncertainties and the short horizontal bars (where visible) show the statistical component.}
\label{fig:plotData3y}
\end{figure}
\section{Results}
\label{sec:results}
The  {\overline  { {\Lambda} } } / { {\Lambda} }  and  {\overline  { {\Lambda} } } / { { K } ^0_{   S}}  production ratios are measured independently for each magnetic field polarity.  These measurements show good consistency after correction for detector acceptance.  Bin-by-bin comparisons in the two-dimensional binning scheme give \chi^2 probabilities for  {\overline  { {\Lambda} } } / { {\Lambda} }  ( {\overline  { {\Lambda} } } / { { K } ^0_{   S}} ) of 3 (18) \% at  { {\sqrts}  = 0.9  { { Te  -0.1em V}} }  and 19 (97) \% at  { {\sqrts}  = 7  { { Te  -0.1em V}} } , with 12 (15) degrees of freedom. The field up  and down  results are therefore combined to maximise statistical significance.  A weighted average is computed such that the result has minimal variance while taking into account the correlations between sources of systematic uncertainty identified in Table~\ref{tab:syst-summary}.  These combined results are shown as a function of  y  in three intervals of  {p_T}  in Fig.~\ref{fig:plotData3y} at  { {\sqrts}  = 0.9  { { Te  -0.1em V}} }  and 7 {  {TeV}}. The ratio  {\overline  { {\Lambda} } } / { { K } ^0_{   S}}  shows a strong  {p_T}  dependence.
\begin{figure}[t!]
\subfigure[]{
\includegraphics[width=0.47 \textwidth]{figs/Fig6a}
\label{fig:plotDataTheory-900-al-y}
}
\subfigure[]{
\includegraphics[width=0.47 \textwidth]{figs/Fig6b}
\label{fig:plotDataTheory-900-al-pt}
}
\subfigure[]{
\includegraphics[width=0.47 \textwidth]{figs/Fig6c}
\label{fig:plotDataTheory-900-ak-y}
}
\subfigure[]{
\includegraphics[width=0.47 \textwidth]{figs/Fig6d}
\label{fig:plotDataTheory-900-ak-pt}
}
\caption{  The ratios  {\overline  { {\Lambda} } } / { {\Lambda} }  and  {\overline  { {\Lambda} } } / { { K } ^0_{   S}}  at  { {\sqrts}  = 0.9  { { Te  -0.1em V}} }  are compared with the predictions of the LHCb MC, Perugia 0 and Perugia NOCR as a function of \subref{fig:plotDataTheory-900-al-y} \& \subref{fig:plotDataTheory-900-ak-y} rapidity and \subref{fig:plotDataTheory-900-al-pt} \& \subref{fig:plotDataTheory-900-ak-pt} transverse momentum.  Vertical lines show the combined statistical and systematic uncertainties and the short horizontal bars (where visible) show the statistical component.}
\label{fig:plotDataTheory-900}
\end{figure}
\begin{figure}
\subfigure[]{
\includegraphics[width=0.47 \textwidth]{figs/Fig7a}
\label{fig:plotDataTheory-7-al-y}
}
\subfigure[]{
\includegraphics[width=0.47 \textwidth]{figs/Fig7b}
\label{fig:plotDataTheory-7-al-pt}
}
\subfigure[]{
\includegraphics[width=0.47 \textwidth]{figs/Fig7c}
\label{fig:plotDataTheory-7-ak-y}
}
\subfigure[]{
\includegraphics[width=0.47 \textwidth]{figs/Fig7d}
\label{fig:plotDataTheory-7-ak-pt}
}
\caption{  The ratios  {\overline  { {\Lambda} } } / { {\Lambda} }  and  {\overline  { {\Lambda} } } / { { K } ^0_{   S}}  at  { {\sqrts}  = 7  { { Te  -0.1em V}} }  compared with the predictions of the LHCb MC, Perugia 0 and Perugia NOCR as a function of \subref{fig:plotDataTheory-7-al-y} \& \subref{fig:plotDataTheory-7-ak-y} rapidity and \subref{fig:plotDataTheory-7-al-pt} \& \subref{fig:plotDataTheory-7-ak-pt} transverse momentum.  Vertical lines show the combined statistical and systematic uncertainties and the short horizontal bars (where visible) show the statistical component.}
\label{fig:plotDataTheory-7}
\end{figure}
\begin{figure}
\subfigure[]{
\includegraphics[width=0.47 \textwidth]{figs/Fig8a}
\label{fig:plotDeltaY-al}
}
\subfigure[]{
\includegraphics[width=0.47 \textwidth]{figs/Fig8b}
\label{fig:plotDeltaY-ak}
}
\caption{  The ratios \subref{fig:plotDeltaY-al}  {\overline  { {\Lambda} } } / { {\Lambda} }  and \subref{fig:plotDeltaY-ak}  {\overline  { {\Lambda} } } / { { K } ^0_{   S}}  from LHCb  are compared at both  { {\sqrts}  = 0.9  { { Te  -0.1em V}} }  (triangles) and 7 TeV (circles) with the published results from STAR \cite{STAR} (squares) as a function of rapidity loss,  {\Delta y}  = y_ {beam} - y.  Vertical lines show the combined statistical and systematic uncertainties and the short horizontal bars (where visible) show the statistical component.}
\label{fig:plotDeltaY}
\end{figure}
Both measured ratios are compared to the predictions of the  { {Pythia}}  6 generator tunes: LHCb MC, Perugia 0 and Perugia NOCR, as functions of  {p_T}  and  y  at  { s }  = 0.9 {  {TeV}} (Fig.~\ref{fig:plotDataTheory-900}) and at  { s }  = 7 {  {TeV}} (Fig.~\ref{fig:plotDataTheory-7}). According to Monte Carlo studies, as discussed in Section~\ref{sec:systematics}, the requirement for a reconstructed primary vertex results in only a small contribution from diffractive events to the selected  { V ^0}   sample, therefore non-diffractive simulated events are used for these comparisons. The predictions of LHCb MC and Perugia 0 are similar throughout.  The ratio  {\overline  { {\Lambda} } } / { {\Lambda} }  is close to Perugia 0 at low  y  but becomes smaller with higher rapidity, approaching Perugia NOCR.  In collisions at  { s }  = 7 {  {TeV}}, this ratio is consistent with Perugia 0 across the measured  {p_T}  range but is closer to Perugia NOCR at  { s }  = 0.9 {  {TeV}}.  The production ratio   {\overline  { {\Lambda} } } / { { K } ^0_{   S}}  is larger in data than predicted by Perugia 0 at both collision energies and in all measurement bins, with the most significant differences observed at high  {p_T} .
To compare results at both collision energies, and to probe scaling violation, both production ratios are shown as a function of rapidity loss,  {\Delta y}  = y_{ {beam}} - y, in Fig.~\ref{fig:plotDeltaY}, where y_{ {beam}} is the rapidity of the protons in the anti-clockwise LHC  beam, which travels along the positive z direction through the detector.  Excellent agreement is observed between results at both  { s }  = 0.9 and 7 {  {TeV}} as well as with results from STAR  at  { s }  = 0.2 {  {TeV}}.  The measured ratios are also consistent with results published by ALICE ~\cite{ALICE} and CMS ~\cite{CMS}.
The combined field up  and down  results are also given in tables in Appendix~\ref{app:final-results}.  Results without applying the model dependent non-prompt correction, as discussed in Section~\ref{sec:analysis}, are shown for comparison in Appendix~\ref{app:un-corrected}.
\section{Conclusions}
The ratio  {\overline  { {\Lambda} } } / { {\Lambda} }  is a measurement of the transport of baryon number from  { {p-p}} collisions to final state hadrons.  There is good agreement with Perugia 0 at low rapidity which is to be expected since the past experimental results used to test this model have focused on that rapidity region.  At high rapidity however, the measurements favour the extreme baryon transport model of Perugia NOCR.  The measured ratio  {\overline  { {\Lambda} } } / { { K } ^0_{   S}}  is significantly larger than predicted by Perugia 0,  {  i.e.} relatively more baryons are produced in strange hadronisation in data than expected, particularly at higher  {p_T} .  Similar results are found at both  { s }  = 0.9 and 7 {  {TeV}}.
When plotted as a function of rapidity loss,  {\Delta y} , there is excellent agreement between the measurements of both ratios at  { s }  = 0.9 and 7 {  {TeV}} as well as with STAR 's results published at 0.2 {  {TeV}}.  The broad coverage of the measurements in  {\Delta y}  provides a unique data set, which is complementary to previous results.  The  { V ^0}   production ratios presented in this paper will help the development of hadronisation models to improve the predictions of Standard Model physics at the LHC  which will define the baseline for new discoveries.  
\section{Tabulated results}
\label{app:final-results}
\begin{table}[h!]
\caption{  The production ratios  {\overline  { {\Lambda} } } / { {\Lambda} }  and  {\overline  { {\Lambda} } } / { { K } ^0_{   S}} , measured at  { {\sqrts}  = 0.9  { { Te  -0.1em V}} } , are quoted in percent with statistical and systematic errors as a function of \subref{tab:val-stat-syst-900-al} \& \subref{tab:val-stat-syst-900-ak} rapidity,  y , and \subref{tab:val-stat-syst-900-pt} transverse momentum,  {p_T}  [  {  {GeV}  -2mu/  -1mu c}].}
\subtable[]{
\begin{tabular}{ccccc}
\toprule
  {\overline  { {\Lambda} } } / { {\Lambda} }   & 2.0<y<2.5 & 2.5<y<3.0 & 3.0<y<3.5 & 3.5<y<4.0 \\
\midrule
0.25< {p_T} <2.50 & 93.4\pm7.2\pm6.1 & 80.0\pm2.5\pm2.5 & 72.7\pm2.0\pm3.3 & 53.9\pm3.1\pm4.0 \\
\midrule
0.25< {p_T} <0.65 & 162.2\pm48.2\pm6.6 & 90.4\pm6.6\pm3.0 & 61.0\pm4.2\pm3.5 & 42.0\pm12.4\pm5.3 \\
0.65< {p_T} <1.00 & 72.3\pm9.7\pm2.5 & 77.2\pm3.9\pm2.4 & 74.6\pm3.3\pm3.9 & 61.7\pm5.6\pm3.6 \\
1.00< {p_T} <2.50 & 90.4\pm11.3\pm2.8 & 74.5\pm4.6\pm2.4 & 75.7\pm3.4\pm3.1 & 48.5\pm3.8\pm2.2 \\
\bottomrule
\end{tabular}
\label{tab:val-stat-syst-900-al}
}
\subtable[]{
\begin{tabular}{ccccc}
\toprule
  {\overline  { {\Lambda} } } / { { K } ^0_{   S}}   & 2.0<y<2.5 & 2.5<y<3.0 & 3.0<y<3.5 & 3.5<y<4.0 \\ 
\midrule
0.25< {p_T} <2.50 & 28.5\pm1.8\pm2.6 & 26.3\pm0.7\pm2.1 & 25.8\pm0.6\pm2.1 & 25.2\pm1.1\pm2.0 \\
\midrule
0.25< {p_T} <0.65 & 19.7\pm3.6\pm2.6 & 21.8\pm1.4\pm2.2 & 18.0\pm1.0\pm1.8 & 15.8\pm3.1\pm2.1 \\
0.65< {p_T} <1.00 & 31.6\pm2.9\pm2.5 & 30.6\pm1.3\pm2.3 & 30.0\pm1.2\pm2.2 & 29.9\pm2.1\pm2.2 \\
1.00< {p_T} <2.50 & 46.3\pm4.5\pm2.9 & 42.9\pm2.1\pm2.5 & 41.3\pm1.6\pm3.2 & 32.3\pm2.0\pm2.6 \\
\bottomrule
\end{tabular}
\label{tab:val-stat-syst-900-ak}
}
\subtable[]{
\begin{tabular}{ccc}
\toprule
 2.0<y<4.0  &  {\overline  { {\Lambda} } } / { {\Lambda} }  &  {\overline  { {\Lambda} } } / { { K } ^0_{   S}}  \\ 
\midrule
0.25< {p_T} <0.50 & 80.6\pm4.6\pm4.0 & 17.7\pm0.8\pm1.7 \\
0.50< {p_T} <0.65 & 73.1\pm3.6\pm3.2 & 21.8\pm0.9\pm1.8 \\
0.65< {p_T} <0.80 & 73.7\pm3.2\pm3.7 & 28.4\pm1.0\pm2.3 \\
0.80< {p_T} <1.00 & 77.5\pm3.2\pm3.7 & 32.3\pm1.2\pm2.4 \\
1.00< {p_T} <1.20 & 70.1\pm3.4\pm2.3 & 36.8\pm1.5\pm2.4 \\
1.20< {p_T} <2.50 & 74.5\pm3.0\pm2.5 & 44.2\pm1.5\pm2.8 \\
\bottomrule
\end{tabular}
\label{tab:val-stat-syst-900-pt}
}
\label{tab:val-stat-syst-900}
\end{table}
\begin{table}[h!]
\caption{  The production ratios  {\overline  { {\Lambda} } } / { {\Lambda} }  and  {\overline  { {\Lambda} } } / { { K } ^0_{   S}} , measured at  { {\sqrts}  = 7  { { Te  -0.1em V}} } , are quoted in percent with statistical and systematic errors as a function of \subref{tab:val-stat-syst-7-al} \& \subref{tab:val-stat-syst-7-ak} rapidity,  y , and \subref{tab:val-stat-syst-7-pt} transverse momentum,  {p_T}  [  {  {GeV}  -2mu/  -1mu c}].}
\subtable[]{
\begin{tabular}{cccccc}
\toprule
  {\overline  { {\Lambda} } } / { {\Lambda} }   & 2.0<y<2.5 & 2.5<y<3.0 & 3.0<y<3.5 & 3.5<y<4.0 & 4.0<y<4.5 \\ 
\midrule
0.15< {p_T} <2.50 & 97.8\pm2.8\pm3.8 & 95.2\pm1.2\pm3.2 & 93.1\pm0.8\pm3.1 & 88.9\pm1.1\pm3.1 & 81.0\pm2.2\pm3.5 \\
\midrule
0.15< {p_T} <0.65 & 87.2\pm16.7\pm11.0 & 95.7\pm1.8\pm3.5 & 94.2\pm1.4\pm3.3 & 87.6\pm2.3\pm3.2 & 90.0\pm12.6\pm4.2 \\
0.65< {p_T} <1.00 & 97.4\pm5.3\pm3.9 & 96.8\pm2.2\pm3.5 & 92.4\pm1.3\pm3.3 & 89.6\pm1.8\pm3.2 & 86.2\pm4.2\pm3.2 \\
1.00< {p_T} <2.50 & 98.7\pm2.9\pm3.4 & 96.6\pm1.8\pm3.3 & 92.8\pm1.5\pm3.2 & 90.3\pm1.7\pm3.2 & 79.2\pm2.8\pm2.9 \\
\bottomrule
\end{tabular}
\label{tab:val-stat-syst-7-al}
}
\subtable[]{
\begin{tabular}{cccccc}
\toprule
  {\overline  { {\Lambda} } } / { { K } ^0_{   S}}   & 2.0<y<2.5 & 2.5<y<3.0 & 3.0<y<3.5 & 3.5<y<4.0 & 4.0<y<4.5 \\ 
\midrule
0.15< {p_T} <2.50 & 29.4\pm0.6\pm2.9 & 27.9\pm0.3\pm2.8 & 27.4\pm0.2\pm2.7 & 27.6\pm0.3\pm2.6 & 28.6\pm0.6\pm2.9 \\
\midrule
0.15< {p_T} <0.65 & 18.2\pm2.7\pm3.0 & 19.1\pm0.3\pm2.6 & 18.5\pm0.2\pm2.5 & 17.5\pm0.4\pm2.5 & 20.7\pm1.5\pm3.0 \\
0.65< {p_T} <1.00 & 32.0\pm1.3\pm3.0 & 32.8\pm0.6\pm3.0 & 31.5\pm0.4\pm2.8 & 29.9\pm0.5\pm2.8 & 32.1\pm1.2\pm2.9 \\
1.00< {p_T} <2.50 & 48.3\pm1.1\pm3.5 & 47.8\pm0.7\pm3.3 & 45.8\pm0.6\pm3.3 & 45.6\pm0.7\pm3.2 & 39.9\pm1.0\pm3.0 \\
\bottomrule
\end{tabular}
\label{tab:val-stat-syst-7-ak}
}
\subtable[]{
\begin{tabular}{ccc}
\toprule
2.0<y<4.5 &  {\overline  { {\Lambda} } } / { {\Lambda} }  &  {\overline  { {\Lambda} } } / { { K } ^0_{   S}}  \\ 
\midrule
0.15< {p_T} <0.50 & 95.4\pm1.4\pm3.4 & 16.2\pm0.2\pm2.4 \\
0.50< {p_T} <0.65 & 93.0\pm1.4\pm3.3 & 23.1\pm0.3\pm2.5 \\
0.65< {p_T} <0.80 & 94.3\pm1.4\pm3.3 & 28.8\pm0.3\pm2.7 \\
0.80< {p_T} <1.00 & 92.3\pm1.3\pm3.2 & 35.1\pm0.4\pm2.8 \\
1.00< {p_T} <1.20 & 93.6\pm1.5\pm3.2 & 41.2\pm0.6\pm3.0 \\
1.20< {p_T} <2.50 & 91.9\pm1.1\pm3.1 & 49.2\pm0.5\pm3.4 \\
\bottomrule
\end{tabular}
\label{tab:val-stat-syst-7-pt}
}
\label{tab:val-stat-syst-7}
\end{table}
\section{Tabulated results before non-prompt correction}
\label{app:un-corrected}
\begin{table}[h!]
\caption{  The production ratios  {\overline  { {\Lambda} } } / { {\Lambda} }  and  {\overline  { {\Lambda} } } / { { K } ^0_{   S}}  without non-prompt corrections at  { {\sqrts}  = 0.9  { { Te  -0.1em V}} }  are quoted in percent with statistical and systematic errors as a function of \subref{tab:no-correction-900-al} \& \subref{tab:no-correction-900-ak}  rapidity,  y , and \subref{tab:no-correction-900-pt} transverse momentum,  {p_T}  [  {  {GeV}  -2mu/  -1mu c}].}
\subtable[]{
\begin{tabular}{ccccc}
\toprule
  {\overline  { {\Lambda} } } / { {\Lambda} }   & 2.0<y<2.5 & 2.5<y<3.0 & 3.0<y<3.5 & 3.5<y<4.0 \\
\midrule
0.25< {p_T} <2.50 & 93.1\pm7.2\pm6.0 & 79.3\pm2.5\pm2.4 & 73.2\pm2.0\pm3.2 & 54.1\pm3.1\pm3.9 \\
\midrule
0.25< {p_T} <0.65 & 163.7\pm48.2\pm6.5 & 89.2\pm6.6\pm2.8 & 61.5\pm4.2\pm3.4 & 41.4\pm12.4\pm5.3 \\
0.65< {p_T} <1.00 & 71.8\pm9.7\pm2.4 & 76.5\pm3.9\pm2.2 & 75.2\pm3.3\pm3.8 & 62.0\pm5.6\pm3.5 \\
1.00< {p_T} <2.50 & 89.9\pm11.3\pm2.7 & 74.2\pm4.6\pm2.3 & 75.7\pm3.4\pm3.0 & 48.5\pm3.8\pm2.1 \\
\bottomrule
\end{tabular}
\label{tab:no-correction-900-al}
}
\subtable[]{
\begin{tabular}{ccccc}
\toprule
  {\overline  { {\Lambda} } } / { { K } ^0_{   S}}   & 2.0<y<2.5 & 2.5<y<3.0 & 3.0<y<3.5 & 3.5<y<4.0 \\ 
\midrule
0.25< {p_T} <2.50 & 28.9\pm1.8\pm2.4 & 27.2\pm0.7\pm1.9 & 26.6\pm0.6\pm1.9 & 25.6\pm1.1\pm1.8 \\
\midrule
0.25< {p_T} <0.65 & 20.7\pm3.6\pm2.4 & 23.0\pm1.4\pm2.0 & 18.9\pm1.0\pm1.6 & 16.3\pm3.1\pm1.9 \\
0.65< {p_T} <1.00 & 31.9\pm2.9\pm2.3 & 31.5\pm1.3\pm2.1 & 31.0\pm1.2\pm2.0 & 30.6\pm2.1\pm2.0 \\
1.00< {p_T} <2.50 & 46.7\pm4.5\pm2.8 & 43.1\pm2.1\pm2.4 & 41.9\pm1.6\pm3.0 & 32.5\pm2.0\pm2.4 \\
\bottomrule
\end{tabular}
\label{tab:no-correction-900-ak}
}
\subtable[]{
\begin{tabular}{ccc}
\toprule
 2.0<y<4.0  &  {\overline  { {\Lambda} } } / { {\Lambda} }  &  {\overline  { {\Lambda} } } / { { K } ^0_{   S}}  \\ 
\midrule
0.25< {p_T} <0.50 & 80.1\pm4.6\pm3.9 & 18.8\pm0.8\pm1.5 \\
0.50< {p_T} <0.65 & 72.9\pm3.6\pm3.1 & 22.9\pm0.9\pm1.6 \\
0.65< {p_T} <0.80 & 73.9\pm3.2\pm3.6 & 29.5\pm1.0\pm2.1 \\
0.80< {p_T} <1.00 & 77.5\pm3.2\pm3.5 & 33.1\pm1.2\pm2.3 \\
1.00< {p_T} <1.20 & 70.1\pm3.4\pm2.1 & 37.2\pm1.5\pm2.2 \\
1.20< {p_T} <2.50 & 74.4\pm3.0\pm2.3 & 44.5\pm1.5\pm2.6 \\
\bottomrule
\end{tabular}
\label{tab:no-correction-900-pt}
}
\label{tab:no-correction-900}
\end{table}
\begin{table}[h!]
\caption{  The production ratios  {\overline  { {\Lambda} } } / { {\Lambda} }  and  {\overline  { {\Lambda} } } / { { K } ^0_{   S}}  without non-prompt corrections at  { {\sqrts}  = 7  { { Te  -0.1em V}} }  are quoted in percent with statistical and systematic errors as a function of \subref{tab:no-correction-7-al} \& \subref{tab:no-correction-7-ak} rapidity,  y , and \subref{tab:no-correction-7-pt} transverse momentum,  {p_T}  [  {  {GeV}  -2mu/  -1mu c}].}
\subtable[]{
\begin{tabular}{cccccc}
\toprule
  {\overline  { {\Lambda} } } / { {\Lambda} }   & 2.0<y<2.5 & 2.5<y<3.0 & 3.0<y<3.5 & 3.5<y<4.0 & 4.0<y<4.5 \\ 
\midrule
0.15< {p_T} <2.50 & 97.3\pm2.8\pm3.6 & 95.1\pm1.2\pm3.1 & 92.7\pm0.8\pm3.0 & 88.6\pm1.1\pm2.9 & 80.9\pm2.2\pm3.4 \\
\midrule
0.15< {p_T} <0.65 & 85.6\pm16.7\pm11.0 & 95.4\pm1.8\pm3.4 & 93.9\pm1.4\pm3.2 & 87.3\pm2.3\pm3.1 & 90.1\pm12.6\pm4.1 \\
0.65< {p_T} <1.00 & 97.5\pm5.3\pm3.8 & 96.5\pm2.2\pm3.4 & 91.8\pm1.3\pm3.1 & 89.5\pm1.8\pm3.1 & 86.2\pm4.2\pm3.0 \\
1.00< {p_T} <2.50 & 98.2\pm2.9\pm3.3 & 96.6\pm1.8\pm3.2 & 92.5\pm1.5\pm3.1 & 90.0\pm1.7\pm3.1 & 79.0\pm2.8\pm2.8 \\
\bottomrule
\end{tabular}
\label{tab:no-correction-7-al}
}
\subtable[]{
\begin{tabular}{cccccc}
\toprule
  {\overline  { {\Lambda} } } / { { K } ^0_{   S}}   & 2.0<y<2.5 & 2.5<y<3.0 & 3.0<y<3.5 & 3.5<y<4.0 & 4.0<y<4.5 \\ 
\midrule
0.15< {p_T} <2.50 & 29.4\pm0.6\pm2.8 & 28.4\pm0.3\pm2.6 & 28.0\pm0.2\pm2.5 & 27.9\pm0.3\pm2.5 & 28.7\pm0.6\pm2.7 \\
\midrule
0.15< {p_T} <0.65 & 18.5\pm2.7\pm2.9 & 20.0\pm0.3\pm2.5 & 19.2\pm0.2\pm2.3 & 17.9\pm0.4\pm2.3 & 21.1\pm1.5\pm2.9 \\
0.65< {p_T} <1.00 & 32.3\pm1.3\pm2.9 & 33.3\pm0.6\pm2.8 & 32.2\pm0.4\pm2.7 & 30.2\pm0.5\pm2.6 & 32.2\pm1.2\pm2.7 \\
1.00< {p_T} <2.50 & 47.9\pm1.1\pm3.3 & 47.5\pm0.7\pm3.2 & 45.7\pm0.6\pm3.2 & 45.6\pm0.7\pm3.1 & 39.5\pm1.0\pm2.8 \\
\bottomrule
\end{tabular}
\label{tab:no-correction-7-ak}
}
\subtable[]{
\begin{tabular}{ccc}
\toprule
 2.0<y<4.5  &  {\overline  { {\Lambda} } } / { {\Lambda} }  &  {\overline  { {\Lambda} } } / { { K } ^0_{   S}}  \\ 
\midrule
0.15< {p_T} <0.50 & 95.0\pm1.4\pm3.2 & 16.9\pm0.2\pm2.3 \\
0.50< {p_T} <0.65 & 92.9\pm1.4\pm3.2 & 23.8\pm0.3\pm2.4 \\
0.65< {p_T} <0.80 & 94.0\pm1.4\pm3.2 & 29.4\pm0.3\pm2.5 \\
0.80< {p_T} <1.00 & 91.9\pm1.3\pm3.1 & 35.5\pm0.4\pm2.7 \\
1.00< {p_T} <1.20 & 93.1\pm1.5\pm3.1 & 41.3\pm0.6\pm2.9 \\
1.20< {p_T} <2.50 & 91.8\pm1.1\pm3.0 & 48.9\pm0.5\pm3.2 \\
\bottomrule
\end{tabular}
\label{tab:no-correction-7-pt}
}
\label{tab:no-correction-7}
\end{table}"""

In [21]:
test_str = r"This {{X}^\bar} is {  {a testing of }} { the usesfulness {}} of {  {this} } command"

In [ ]:


print(new_str)

{{X}
{  {a testing of }
{  {this}
This {{X}^\bar} is {a testing of } { the usesfulness {}} of {this} command


In [7]:
print(new_str)

\section{Introduction}
\label{sec:introduction}
While the underlying interactions of hadronic collisions and hadronisation are understood within the Standard Model, exact computation of the processes governed by QCD are difficult due to the highly non-linear nature of the strong force.  In the absence of full calculations, generators based on phenomenological models have been devised and optimised, or ``tuned'', to accurately reproduce experimental observations.  These generators predict how Standard Model physics will behave at the LHC  and constitute the reference for discoveries of New Physics effects.  
Strange quark production is a powerful probe for hadronisation processes at  {p-p} colliders since protons have no net strangeness.  Recent experimental results in the field have been published by STAR ~\cite{STAR} from RHIC   {p-p} collisions at  { s }  = 0.2 {TeV} and by ALICE ~\cite{ALICE}, CMS ~\cite{CMS} and LHCb ~\cite{KsCrossSection} from LHC   {p-p} collisions at  { s }  = 0